In [3]:
import numpy as np
import graphviz

In [22]:
!dot -Tpng -O diagraph.txt

![diagraph.png](./diagraph.txt.png)

In [ ]:
# Define some parameters of the simulation
sim_par = {'N_sol': 20}

In [ ]:
# Define the variables to study in the model

par = {'S': 2,  # number of intra neurons
       'Theta': 0.} # threshold value

# Define the connectivity matrix J0_iJ0 (I cannot put it in the par dictionary because it depends on S)
J0 = np.ones((2+par['S']+1,2+par['S']+1)) # Initialized all to one, so all connnected

# First, impose on J0 that the elements on the diagonal must be zero
for j in range(len(J0)):
    J0[j,j] = 0.

# Impose on the connectiviy matrix the fact that nuerons on the same layer cannot be connected (now I do it by hand, surely it is better to create a class)
J0[0,1] = 0.
J0[1,0] = 0.
for i in range(2, 2 + par['S']):
    for j in range(2, 2 + par['S']):
        if i != j:
            J0[i, j] = 0.

print('J0:')
print(J0)


J0:
[[0. 0. 1. 1. 1.]
 [0. 0. 1. 1. 1.]
 [1. 1. 0. 0. 1.]
 [1. 1. 0. 0. 1.]
 [1. 1. 1. 1. 0.]]


In [29]:
a = np.array([[2,3],[4,5]])
print(a)
a[0,1]

[[2 3]
 [4 5]]


3

As a convention, J_ij is the link that goes from node j to node i

In [ ]:
# I try to create a class for the NN
class Network:
    """
    Class for a generic neural network, defined by 
    - 2 input neurons, 1 output neuron
    - S: # of intra neurons.
    - Theta: threshold value.
    - J: connectivity matrix.
    """
    
    def __init__(self, par, J):
        self.S = par['S']
        self.Theta = par['Theta']
        self.J = J
        self.activation = np.heaviside()
        self.neurons = np.zeros(2+self.S+1) # container for storing the values of the neurons
        
        
    def output(self,input):
        self.neurons[0:2] = input   # set the value of the first two neurons to the input value
        
        for neuron in range(1,2+self.S):    # compute the value of the intra neurons by feed forward
            self.neurons[neuron] = self.J[0,neuron] * self.neurons[0] + self.J[1,neuron] * self.neurons[1]   # I refer to the upper triangle of the matrix J
        
        for neuron in range(0,2+self.S):  # compute the value of the ouput
        
    def fitness(self,target):
        
        squared_dist = (target - )
    

For this model I don't need to define the space grid, I just need to generate the solutions and then execute the algorithm. I just need to make sure that they stay in the range.

I try first one loop:

In [ ]:
# Generate a set of parameters
Sk_range = [1,2,3,4,5,6,7,8,9]  # define the possible value for Sk
Sk = Sk_range[np.random.randint(len(Sk_range))]
Thetak = np.random.uniform(-1,+1,1)
Jk = J0 * np.random.uniform(-1,+1,25).reshape((5,5))

In [ ]:
# Generate the solutions
solutions = []
for _ in range(sim_par['N_sol']):
    # Generate parameters
    par['S'] = Sk_range[np.random.randint(len(Sk_range))]
    par['Theta'] = np.random.uniform(-1,+1,1)
    Jk = J0 * np.random.uniform(-1,+1,25).reshape((5,5))
    # Create the network associated with the generated parameters and add it to the solutions list
    solutions.append(Network(par,Jk))
    